In [1]:
import datetime as dt
import pennylane as qml
from pennylane import numpy as np
import math

# Fixed parameters

In [2]:
samplesize = 5000
order = 2
mean_size = 20

# Variable parameters (qubits and layers)

In [3]:
#List of number of qubits
qubits = [3]

#List of number of layers
layers = [1]

# Multiple layer fidelities 'order' moment calculations

In [4]:
def designtest(order, mean_size, qubits, samples, circuit, nlayers):
    express_mean = 0
    express_mean_sqr = 0
    for j in range(mean_size):
        fidel_moment = 0
        for i in range(samples):
            #angles1 = np.array([np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)],
            #             requires_grad=True)
            #angles2 = np.array([np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)],
            #             requires_grad=True)

            angles1 = np.array([ [[np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)], 
                                [np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)]] 
                                for j in range(nlayers)], requires_grad=True)

            angles2 = np.array([ [[np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)], 
                                [np.random.uniform(low=-np.pi, high=np.pi) for i in range(wires)]] 
                                for j in range(nlayers)], requires_grad=True)

            state1 = circuit(angles1)
            state2 = circuit(angles2)

            F = np.abs( np.dot(state1, state2) )**2

            fidel_moment = fidel_moment + F**order

        fidel_moment = fidel_moment/samples

        expressib = fidel_moment - (math.factorial(order) * math.factorial(2**qubits-1))/(math.factorial(order
                                                                                                        +2**qubits-1))
        expressib_sqr = expressib**2
        
        express_mean = express_mean + expressib
        express_mean_sqr = express_mean_sqr + expressib_sqr
        
    express_mean = express_mean/mean_size
    express_mean_sqr = express_mean_sqr/mean_size
    express_stdev = np.sqrt(express_mean_sqr - express_mean**2)
    return(express_mean, express_stdev)

# Code that executes the different conditions

In [5]:
now0 = dt.datetime.now() #monitorar tempo de início
now0 = now0.strftime("%Y-%m-%d %H:%M:%S")


for wires in qubits:
    dev = qml.device("default.qubit", wires=wires, shots=None)
        
    
    for nlayers in layers:
        
        #No connections
        def layer_noconnec(rots):
            for i in range(wires):
                qml.RX(rots[0][i], wires=[i])
                qml.RY(rots[1][i], wires=[i])
        
        def noconnec(rotations):
            qml.layer(layer_noconnec, nlayers, rotations)
            return qml.state()

        noconnec_circuit = qml.QNode(noconnec, dev)
        
        
        
        #Linear
        def layer_linear(rots):
            for i in range(wires):
                qml.RX(rots[0][i], wires=[i])
                qml.RY(rots[1][i], wires=[i])
            qml.broadcast(qml.CNOT, wires=range(wires), pattern="chain")

        def linear(rotations):
            qml.layer(layer_linear, nlayers, rotations)
            return qml.state()

        linear_circuit = qml.QNode(linear, dev)
        
        
        
        #Ring
        def layer_ring(rots):
            for i in range(wires):
                qml.RX(rots[0][i], wires=[i])
                qml.RY(rots[1][i], wires=[i])
            qml.broadcast(qml.CNOT, wires=range(wires), pattern="chain")
            qml.CNOT(wires=[wires-1,0])
    
        def ring(rotations):
            qml.layer(layer_ring, nlayers, rotations)
            return qml.state()

        ring_circuit = qml.QNode(ring, dev)
        
        
        
        noconnec_design, noconnec_design_stdev  = designtest(order, mean_size, wires, samplesize, noconnec_circuit, nlayers)
        linear_design, linear_design_stdev = designtest(order, mean_size, wires, samplesize, linear_circuit, nlayers)
        ring_design, ring_design_stdev = designtest(order, mean_size, wires, samplesize, ring_circuit, nlayers)

        
        
        f = open("designtest_withStDev_ansatz1_{0}design_qubits{1}_layers{2}_samples{3}.txt"
                 .format(order,wires,nlayers, samplesize), "w")

        print('Circuit,Designtest,Designtest_stdev', file = f)
        print('No connections,', noconnec_design,',',noconnec_design_stdev, file = f)
        print('Linear,', linear_design,',',linear_design_stdev, file = f)
        print('Ring,', ring_design,',',ring_design_stdev, file = f)
        
        f.close()
        
        print('Execution: qubits=',wires,'; layers=', nlayers)
        print()
        
        
now1 = dt.datetime.now() #monitorar tempo de início
now1 = now1.strftime("%Y-%m-%d %H:%M:%S")


print("Tempo inicial: ")
print(now0)
print("Tempo final:")
print(now1)

Execution: qubits= 3 ; layers= 1

Tempo inicial: 
2023-11-18 13:40:04
Tempo final:
2023-11-18 14:32:21
